In [65]:
import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import copy
import numpy as np
from torchvision import datasets, transforms
import torch

import math

from utils.sampling import mnist_iid, mnist_noniid, cifar_iid
from utils.options import args_parser
from models.Update import LocalUpdate
from models.Nets import MLP, CNNMnist, CNNCifar, LeNet, CNNMnist2
from models.Fed import FedAvg
from models.Fed import FedQAvg, Quantization, Quantization_Finite, my_score, my_score_Finite
from models.test import test_img

%load_ext autoreload
%autoreload 2

import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import copy
import numpy as np
from torchvision import datasets, transforms
import torch

from utils.sampling import mnist_iid, mnist_noniid, cifar_iid
from utils.options import args_parser
from models.Update import LocalUpdate
from models.Nets import MLP, CNNMnist, CNNCifar, LeNet, CNNMnist2
from models.Fed import FedAvg
from models.Fed import FedQAvg, Quantization, Quantization_Finite, my_score, my_score_Finite
from models.test import test_img
from torch.utils.data import DataLoader, Dataset
from utils.functions import *

%load_ext autoreload
%autoreload 2

class DatasetSplit(Dataset):
    def __init__(self, dataset, idxs):
        self.dataset = dataset
        self.idxs = list(idxs)

    def __len__(self):
        return len(self.idxs)

    def __getitem__(self, item):
        image, label = self.dataset[self.idxs[item]]
        return image, label

class my_argument:    
    epochs = 200    #"rounds of training"
    num_users = 15  # "number of users: N"
    num_partition = 2 # "number of users: K"
    frac = 0.5 #"the fraction of clients: C"
    local_ep = 1 #"the number of local epochs: E"
    local_bs = 200 #"local batch size: B"
    bs=200 #"test batch size"
    lr=0.01 #"learning rate"
    momentum=0.5 # "SGD momentum (default: 0.5)"
    split='user' # "train-test split type, user or sample"
    opt='ADAM'
    loss='Custom' # 'Custom' or 'Default'

    # model arguments
    model = 'cnn'
    kernel_num=9 #, help='number of each kind of kernel')
    kernel_sizes='3,4,5' #  help='comma-separated kernel size to use for convolution')
    norm='None' #, help="batch_norm, layer_norm, or None")
    num_filters=32 #, help="number of filters for conv nets")
    max_pool='True' #help="Whether use max pooling rather than strided convolutions")

    # other arguments
    dataset='mnist' #, help="name of dataset")
    iid=1
    num_classes=10#, help="number of classes")
    num_channels=1#, help="number of channels of imges")
    gpu=1#, help="GPU ID, -1 for CPU")
    stopping_rounds=10#, help='rounds of early stopping')
    verbose='False'#, help='verbose print')
    seed=1#, help='random seed (default: 1)')
    
args = my_argument()

args.device = torch.device('cuda:{}'.format(args.gpu) if torch.cuda.is_available() and args.gpu != -1 else 'cpu')

# load dataset and split users
trans_mnist = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
dataset_train = datasets.MNIST('../data/mnist/', train=True, download=True, transform=trans_mnist)
dataset_test = datasets.MNIST('../data/mnist/', train=False, download=True, transform=trans_mnist)

dict_users = mnist_iid(dataset_train, args.num_partition)

encoding_input_array_np = np.empty((len(dataset_train),28*28))
encoding_label_array_np = np.empty((len(dataset_train),args.num_classes))
print("size of X:" ,encoding_input_array_np.shape)
print("size of Y:" ,encoding_label_array_np.shape)

Size_submatrices = int(60000/args.num_partition)

X_in = np.empty((args.num_partition, Size_submatrices, 28*28))
Y_in = np.empty((args.num_partition, Size_submatrices, 10))

for i in range(args.num_partition):
    
    stt_pos = i*Size_submatrices
    end_pos = (i+1)*Size_submatrices
#     print(i,stt_pos,end_pos)
    Temp_train = DataLoader(DatasetSplit(dataset_train, dict_users[i]), batch_size=Size_submatrices, shuffle=True)
    
    for batch_idx, (images, labels) in enumerate(Temp_train):
        
        images_np = images.detach().cpu().numpy()
        encoding_input_array_np[stt_pos:end_pos,:] = np.reshape(images_np, (Size_submatrices,28*28))
#         print(encoding_input_array_np[stt_pos:end_pos,:].shape)

        onehot_labels = torch.nn.functional.one_hot(labels,num_classes=args.num_classes)
        labels_np = onehot_labels.detach().cpu().numpy()
#         print(labels_np.shape)
        encoding_label_array_np[stt_pos:end_pos,:] = labels_np

        X_in[i,:,:] = np.reshape(images_np, (Size_submatrices,28*28))
        Y_in[i,:,:] = labels_np
# print(labels_np[0:10,:])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
size of X: (60000, 784)
size of Y: (60000, 10)


In [66]:
print(np.shape(X_in))

(2, 30000, 784)


# 1. Compute the true gradient

In [67]:
net_glob_init = CNNMnist2(args=args)
net_glob_init.cuda()
net_glob_init.train()

# copy weights
w_glob_init = net_glob_init.state_dict()

In [68]:
use_cuda = torch.cuda.is_available()
print(use_cuda)
args.device = torch.device("cuda" if use_cuda else "cpu")
print(args.device)

True
cuda


In [71]:
from models.Update import LocalUpdate_with_BACC
from models.Fed import FedAvg_with_BACC_Dec

from utils.functions import *
import math

K = 2
T = 0
N_epochs = 3
N_trials = 1

N_array = [10]
B_array = [1]

N = N_array[0]

loss_test_arr_v1 = np.zeros((len(N_array),len(B_array),N_trials,N_epochs))
acc_test_arr_v1  = np.zeros((len(N_array),len(B_array),N_trials,N_epochs))

for trial_idx in range(N_trials):

    net_glob = CNNMnist2(args=args)
    net_glob.cuda()
    net_glob.train()
    
    net_glob.load_state_dict(w_glob_init)

    # copy weights
    w_glob = net_glob.state_dict()
    
    for iter in range(N_epochs): #args.epochs
        
        prev_net_glob = CNNMnist2(args=args)
        prev_net_glob.cuda()
        prev_net_glob.load_state_dict(w_glob)
        
        w_locals, loss_locals = [], []
        m = K
        idxs_users = np.random.choice(range(K), m, replace=False)
        idxs_users = np.sort(idxs_users)
        
        for idx in idxs_users:
    #         print(idx)
#             local = LocalUpdate(args=args, dataset=dataset_train, idxs=dict_users[idx])
# w, loss = local.train(net=copy.deepcopy(net_glob).to(args.device))
            local = LocalUpdate_with_BACC(args=args, dataset=X_in[idx,:,:], label=Y_in[idx,:,:])
        
#             print(local.ldr_train)
            w, loss = local.train(net=copy.deepcopy(net_glob).cuda())
            w_locals.append(copy.deepcopy(w))
            loss_locals.append(copy.deepcopy(loss))
        # update global weights
        w_glob = FedAvg(w_locals)

        # copy weight to net_glob
        net_glob.load_state_dict(w_glob)

        # print loss
        loss_avg = sum(loss_locals) / len(loss_locals)

#         loss_train.append(loss_avg)

        acc_test, loss_test = test_img(net_glob, dataset_test, args)

        if iter % 1 ==0:
            print('Round {:3d}, Average loss {:.3f} Test accuracy {:.3f}'.format(iter, loss_avg,acc_test))
        #print(loss_train)


Test set: Average loss: 2.1437 
Accuracy: 2483/10000 (24.83%)

Round   0, Average loss 2.071 Test accuracy 24.830

Test set: Average loss: 0.5204 
Accuracy: 7863/10000 (78.63%)

Round   1, Average loss 0.718 Test accuracy 78.630

Test set: Average loss: 0.0459 
Accuracy: 9849/10000 (98.49%)

Round   2, Average loss 0.150 Test accuracy 98.490


In [37]:
print(len(dict_users[0]))

20000


In [28]:
w_init = prev_w_glob
w_true = w_locals

In [30]:
net_glob_tmp = CNNMnist2(args=args)
net_glob_tmp.cuda()
net_glob_tmp.load_state_dict(w_init)
print(test_img(net_glob_tmp, dataset_test, args))

net_glob_tmp = CNNMnist2(args=args)
net_glob_tmp.cuda()
net_glob_tmp.load_state_dict(w_true[0])
print(test_img(net_glob_tmp, dataset_test, args))

net_glob_tmp = CNNMnist2(args=args)
net_glob_tmp.cuda()
net_glob_tmp.load_state_dict(w_true[1])
print(test_img(net_glob_tmp, dataset_test, args))

print(test_img(net_glob, dataset_test, args))


Test set: Average loss: 1.9355 
Accuracy: 5806/10000 (58.06%)

(tensor(58.0600), 1.9355282196044923)

Test set: Average loss: 0.8423 
Accuracy: 6599/10000 (65.99%)

(tensor(65.9900), 0.8423189865112305)

Test set: Average loss: 1.5206 
Accuracy: 3748/10000 (37.48%)

(tensor(37.4800), 1.5206392913818358)

Test set: Average loss: 1.9355 
Accuracy: 5806/10000 (58.06%)

(tensor(58.0600), 1.9355282196044923)


In [21]:
print(w_true[0])
print(w_true[1])

print(torch.dist(w_true[0],w_true[1], 2))



OrderedDict([('conv1.weight', tensor([[[[-1.3603e-01, -1.2559e-01, -1.9498e-01, -7.6227e-02, -1.8486e-01],
          [-1.4288e-01,  8.3199e-02,  4.3605e-02,  1.9782e-02, -4.6460e-02],
          [-1.4529e-02, -1.1040e-01, -2.6452e-01, -1.2712e-01,  9.6407e-02],
          [-8.5253e-02, -8.0197e-03,  7.9901e-02,  4.3853e-02, -2.1372e-01],
          [ 3.7502e-02, -2.8371e-01,  3.5868e-02, -1.1850e-01, -2.1351e-01]]],


        [[[ 6.4013e-02, -2.7716e-01,  9.0304e-02, -3.8140e-02, -1.3866e-01],
          [-2.6533e-01,  6.4480e-02, -1.5361e-01, -1.0618e-01, -1.5836e-01],
          [ 7.1878e-02,  9.5403e-02, -3.6925e-02, -1.1346e-01, -4.6461e-02],
          [ 4.8427e-02,  8.3709e-02, -1.9632e-01, -1.6786e-01,  6.8837e-02],
          [-1.4388e-02, -1.8442e-01, -2.4330e-01,  6.2458e-02, -3.2144e-02]]],


        [[[-1.5117e-01, -8.0029e-03,  2.9809e-02, -1.4247e-01, -2.1474e-02],
          [-1.8055e-01, -1.9644e-01, -9.5055e-02,  4.2124e-03,  8.4319e-02],
          [ 1.2163e-01, -9.8152e-02, -

TypeError: dist(): argument 'input' (position 1) must be Tensor, not collections.OrderedDict

In [17]:
def check_distance(w1,w2):
    
    w_tmp = w1
    
    for k in w_tmp.keys():
        tmp1 = w1[k].cpu().detach().numpy()
        tmp2 = w2[k].cpu().detach().numpy()
        cur_shape = tmp1.shape
        _d = np.prod(cur_shape)
        
        print(k,cur_shape)
        
        tmp1 = np.reshape(tmp1,(1,_d))
        tmp2 = np.reshape(tmp2,(1,_d))
        
        dist = tmp1 - tmp2
        
check_distance(w_true[0],w_true[1])

conv1.weight (32, 1, 5, 5)
conv1.bias (32,)
conv2.weight (64, 32, 5, 5)
conv2.bias (64,)
fc1.weight (10, 1024)
fc1.bias (10,)


# K=2, T=0, Check the Continuity of CNN

In [ ]:
from models.Update import LocalUpdate_with_BACC
from models.Fed import FedAvg_with_BACC_Dec

from utils.functions import *
import math

K = 3
T = 4
sigma = 1
Noise_Alloc = [0,2,4,6]

Signal_Alloc = []
for i in range(K+T):
    if i not in Noise_Alloc:
        Signal_Alloc.append(i)


j_array = np.array(range(K+T))
alpha_array = np.cos((2*j_array+1)*math.pi/(2*(K+T))) #np.cos((2*j_array+1)*math.pi/(2*K))
# print("alpha_array: ",alpha_array,'\n')

B = 0.5


N_array = [3,4,5,6,7,8]
B_array = [0.5, 1, 2]

N_trials = 1
N_epochs = 30



loss_test_arr_v1 = np.zeros((len(N_array),len(B_array),N_trials,N_epochs))
acc_test_arr_v1  = np.zeros((len(N_array),len(B_array),N_trials,N_epochs))

for N_idx in range(len(N_array)):
    
    N = N_array[N_idx]
    
    
           
        
    # print("alpha_array: ",alpha_array,'\n')
    
    
    for B_idx in range(len(B_array)):
        
        B = B_array[B_idx]
        z_array = []
        while(len(z_array)<N):
            z_tmp = np.random.uniform(-1,1,1)
            MIS_tmp = MutualInformationSecurity(alpha_array[Signal_Alloc], alpha_array[Noise_Alloc],[z_tmp], 1,sigma)
            if MIS_tmp < B and MIS_tmp > 0.1:
                z_array.append(z_tmp[0])
        
        z_array = np.sort(z_array)
        print('z_array:',z_array)
        
        sigma = 1 #sigma_array[sigma_idx]
        
        _Noise_label = np.ones((20000*T,10)) * 0.1

        X_tilde,a,b = BACC_Enc_Data_v3(encoding_input_array_np, N, K, T, sigma, alpha_array, z_array, _Noise_Alloc = Noise_Alloc) #BACC_Enc(encoding_input_array_np, alpha_array, z_array)
        y_tilde,a,b = BACC_Enc_Data_v3(encoding_label_array_np, N, K, T, sigma, alpha_array, z_array, _Noise_Alloc = Noise_Alloc, _Noise = _Noise_label, is_predefined_noise=True) #BACC_Enc(encoding_label_array_np, alpha_array, z_array)

        m = N # m is the number of received result @ master
    #     print('number of results:',m)

        for trial_idx in range(N_trials):
            print('(T, sigma)=',T,sigma,') ',trial_idx,'-th Trial!!')

            net_glob = CNNMnist2(args=args)
            net_glob.cuda()
            net_glob.train()

            # copy weights
            w_glob = net_glob.state_dict()

            for iter in range(N_epochs): #args.epochs
                w_locals, loss_locals = [], []
                idxs_users = np.random.choice(range(N), m, replace=False)
                idxs_users = np.sort(idxs_users)
                print('selected users:',idxs_users)
                
                coded_net = BACC_Enc_Model_withNoise_v3(net_glob.cuda(), N, K, T, 0.00, alpha_array, z_array, _Noise_Alloc=Noise_Alloc)

                dec_z_array = []
                for idx in idxs_users: #for idx in range(N):
            #         print(idx)
                    local = LocalUpdate_with_BACC(args=args, dataset=X_tilde[idx,:,:], label=y_tilde[idx,:,:])
                    w, loss = local.train(net=copy.deepcopy(coded_net[idx]).cuda())
                    w_locals.append(copy.deepcopy(w))
                    loss_locals.append(copy.deepcopy(loss))

                    dec_z_array.append(z_array[idx])


                # update global weights
                #w_glob = FedAvg(w_locals)
                w_glob = FedAvg_with_BACC_Dec(w_locals, alpha_array[Signal_Alloc], dec_z_array)

                # copy weight to net_glob
                net_glob.load_state_dict(w_glob)

                # print loss
            #     acc_train, loss_train = test_img(net_glob, dataset_train, args)

            #     loss_train_arr.append(loss_train)

                acc_test, loss_test = test_img(net_glob, dataset_test, args)
                acc_test_arr_v1[N_idx][B_idx][trial_idx][iter] = acc_test
                loss_test_arr_v1[N_idx][B_idx][trial_idx][iter] = loss_test
                if iter % 1 ==0:
                    print('Round {:3d}, Average loss {:.3f} Test accuracy {:.3f}'.format(iter, loss_test, acc_test))
                #print(loss_train)

In [73]:
def PI(vals):  # upper-case PI -- product of inputs
    accum = 1
    for v in vals:
        accum = accum*v
    return accum

def gen_Lagrange_coeffs(alpha_s,beta_s,is_K1=0):
    if is_K1==1:
        num_alpha = 1
    else:
        num_alpha = len(alpha_s)
    U = np.zeros((num_alpha, len(beta_s)))
#         U = [[0 for col in range(len(beta_s))] for row in range(len(alpha_s))]
    #print(alpha_s)
    #print(beta_s)
    for i in range(num_alpha):
        for j in range(len(beta_s)):
            cur_beta = beta_s[j];

            den = PI([cur_beta - o   for o in beta_s if cur_beta != o])
            num = PI([alpha_s[i] - o for o in beta_s if cur_beta != o])
            U[i][j] = num/den
            # for debugging
            # print(i,j,cur_beta,alpha_s[i])
            # print(test)
            # print(den,num) 
    return U

K=2
T=3

j_array = np.array(range(K+T))
alpha_array = np.cos((2*j_array+1)*math.pi/(2*(K+T))) #np.cos((2*j_array+1)*math.pi/(2*K))

z_array = np.array([-0.9, -0.81, -0.22, -0.16, 0.16, 0.22, 0.81, 0.9])

print(alpha_array)
print(z_array)
U = gen_Lagrange_coeffs(z_array,alpha_array,)

print(U)

[ 9.51056516e-01  5.87785252e-01  6.12323400e-17 -5.87785252e-01
 -9.51056516e-01]
[-0.9  -0.81 -0.22 -0.16  0.16  0.22  0.81  0.9 ]
[[-0.02110667  0.06875027 -0.14048     0.32761308  0.76522331]
 [-0.03509327  0.11575292 -0.24690493  0.72811427  0.438131  ]
 [ 0.04724939 -0.1793302   0.81389619  0.39387193 -0.07568732]
 [ 0.04003709 -0.15573889  0.89969715  0.27223764 -0.05623299]
 [-0.05623299  0.27223764  0.89969715 -0.15573889  0.04003709]
 [-0.07568732  0.39387193  0.81389619 -0.1793302   0.04724939]
 [ 0.438131    0.72811427 -0.24690493  0.11575292 -0.03509327]
 [ 0.76522331  0.32761308 -0.14048     0.06875027 -0.02110667]]
